In [27]:
# importar las librerias necesarias
import pandas as pd
import numpy as np

In [28]:
# ingestar la data y la mirar
df = pd.read_csv(r'/home/frangr94/Fran/HenryData/pi_mlops/Dataset/movies_dataset.csv')
df.head()

/tmp/ipykernel_41681/4117005002.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/home/frangr94/Fran/HenryData/pi_mlops/Dataset/movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [29]:
# tomar solo los datos numericos de 'id': no son nulos pero los datos no estan utilizables
df = df.drop(df.loc[~df['id'].apply(pd.to_numeric, errors='coerce').notna()].index)
df['id']=df['id'].astype(int)

In [30]:
# rellenar na con 0 en las columnas requeridas
df.revenue.fillna(0, inplace=True)

In [31]:
# en budget hay valores no numericos --> cambiar a na y reemplazar por 0
df = df.drop(df.loc[~df['budget'].apply(pd.to_numeric, errors='coerce').notna()].index)
df.budget.fillna(0, inplace=True)
df['budget']=df['budget'].astype(float)

In [32]:
# eliminar los valores nulos del campo release date
df.dropna(subset=['release_date'],inplace=True)

In [33]:
# cambiar columnas de fechas al formato AAAA-mm-dd
df.release_date.head() # ya estan

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [34]:
# crear la columna release_year
df['release_date']=pd.to_datetime(df.release_date)
df['release_year'] = df['release_date'].dt.year

# crear la columna return
df['return'] = np.where(df['budget']!=0,df.revenue/df.budget,0)

# eliminar columnas innecesarias
df.drop(columns=['video','imdb_id','adult','poster_path','homepage'],inplace=True)

In [35]:
# desanidar campos usando AST --> no voy a crear una funcion dado que las llaves de los diccionarios pueden variar dependiendo del campo
import ast

# generos:
lista_generos=[]
for i in df['genres']:
    lista_generos.append(ast.literal_eval(i)) # compara el string con estructuras de python-> si es, devuelve un objeto del tipo detectado, se introduce en una lista

generos_peliculas=[]
for i in lista_generos: # se itera por cada lista y se toman las llaves del diccionario que sean necesarias, introduciendo los valores en en una lista
    generos=[]
    for j in i:
        g=j.get('name')
        generos.append(g)
    generos_peliculas.append(generos)

df['genres_unn'] = generos_peliculas # se crea un nuevo campo _unn (unnested), se elimina el campo anidado
df.drop(columns='genres',inplace=True)

In [36]:
# spoken languages
lista_lang=[]
for i in df['spoken_languages']:
    lista_lang.append(ast.literal_eval(i))

langs_peliculas=[]
for i in lista_lang:
    langs=[]
    for j in i:
        g=j.get('iso_639_1')
        langs.append(g)
    langs_peliculas.append(langs)

df['langs_unn'] = langs_peliculas
df.drop(columns='spoken_languages',inplace=True)

In [37]:
# collection
df['belongs_to_collection'].fillna('{}',inplace=True)
lista_coll=[]
for i in df['belongs_to_collection']:
    lista_coll.append(ast.literal_eval(i)) 

coll_peliculas=[]
for i in lista_coll:
    if 'name' in i.keys():
        coll_peliculas.append(i['name'])
    else:
        coll_peliculas.append('no')

df['collection_unn']=coll_peliculas    
df.drop(columns='belongs_to_collection',inplace=True)

In [38]:
# production companies
lista_prod=[]
for i in df['production_companies']:
    lista_prod.append(ast.literal_eval(i))
prod_peliculas=[]
for i in lista_prod:
    langs=[]
    for j in i:
        g=j.get('name')
        langs.append(g.strip('''""'''))
    prod_peliculas.append(langs)

df['prod_companies'] = prod_peliculas
df.drop(columns='production_companies',inplace=True)

In [39]:
# production_countries
lista_country=[]
for i in df['production_countries']:
    lista_country.append(ast.literal_eval(i))
country_peliculas=[]
for i in lista_country:
    countries=[]
    for j in i:
        g=j.get('iso_3166_1')
        countries.append(g)
    country_peliculas.append(countries)

df['prod_countries_unn'] = country_peliculas
df.drop(columns='production_countries',inplace=True)


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   budget              45376 non-null  float64       
 1   id                  45376 non-null  int64         
 2   original_language   45365 non-null  object        
 3   original_title      45376 non-null  object        
 4   overview            44435 non-null  object        
 5   popularity          45376 non-null  object        
 6   release_date        45376 non-null  datetime64[ns]
 7   revenue             45376 non-null  float64       
 8   runtime             45130 non-null  float64       
 9   status              45296 non-null  object        
 10  tagline             20398 non-null  object        
 11  title               45376 non-null  object        
 12  vote_average        45376 non-null  float64       
 13  vote_count          45376 non-null  float64       


In [ ]:
#df.to_csv('data.csv')
